In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'train', 'train.csv', 'sample_submission.csv']


In [28]:
import glob
import pandas as pd
import numpy as np
data = pd.read_csv('../input/train.csv')
data['path'] = '../input/train/train/'
data['path'] = data[['path','id']].apply(lambda x: "".join(x), axis=1)
data.drop(['id'],axis=1,inplace=True)
data.head()

,has_cactus,path
0,1,../input/train/train/0004be2cfeaba1c0361d39e2b...
1,1,../input/train/train/000c8a36845c0208e833c79c1...
2,1,../input/train/train/000d1e9a533f62e55c289303b...
3,1,../input/train/train/0011485b40695e9138e92d0b3...
4,1,../input/train/train/0014d7a11e90b62848904c141...


In [29]:
all_images = glob.glob('../input/train/train/*')
print(len(all_images), data.shape)

IMG_DIM = (30,30,3)

from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
imgs = np.array([ img_to_array( load_img(file, target_size = IMG_DIM) ) for file in data.path.values ])
labels = data.has_cactus.values


print(imgs.shape,labels.shape)

17500 (17500, 2)
(17500, 30, 30, 3) (17500,)


In [30]:
from sklearn.model_selection import train_test_split
X_train,X_val,Y_train,Y_val = train_test_split(imgs,labels,test_size=0.3, stratify=labels)

train_datagen = ImageDataGenerator(rescale=1./255, 
                                   zoom_range=0.3, 
                                   rotation_range=50,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, Y_val, batch_size=32)

In [31]:
from keras.layers import Input,Conv2D,Dense,Dropout, MaxPooling2D, Flatten
from keras import optimizers
from keras.models import Model

#Input Layer
inp = Input(IMG_DIM)

#1st Conv
conv_1  = Conv2D( 64, kernel_size=(2,2), activation='relu')(inp)
pool_1 = MaxPooling2D(pool_size=(2,2))(conv_1)

#2nd Conv
conv_2 = Conv2D( 32, kernel_size=(2,2), activation='relu')(pool_1)
pool_2 = MaxPooling2D(pool_size=(2,2))(conv_2)

#flatten
flatten = Flatten()(pool_2)
dropout_1 = Dropout(0.3)(flatten)

#1st Dense
dense_1 = Dense(512, activation='relu')(dropout_1)
dropout_2 = Dropout(0.3)(dense_1)

#2nd Dense
dense_2 = Dense(64,activation='relu')(dropout_2)
dropout_3 = Dropout(0.2)(dense_2)

#output
output = Dense(1, activation='sigmoid')(dropout_3)

model = Model(inp,output)

model.compile( loss='binary_crossentropy', optimizer=optimizers.Adam(lr=1e-4), metrics=['accuracy'] )

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 30, 30, 3)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 29, 29, 64)        832       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 13, 13, 32)        8224      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 1152)              0         
__________

In [32]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

model_checkpoint  = ModelCheckpoint('model_best_checkpoint.h5', save_best_only=True, monitor='val_acc', mode='max', verbose=2)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, mode='min')

callback_list = [model_checkpoint]

history = model.fit_generator(train_generator, steps_per_epoch=150, epochs=400,
                              validation_data=val_generator, validation_steps=50, 
                              verbose=1,callbacks=callback_list)

Epoch 1/400
150/150 [==============================] - 4s 28ms/step - loss: 0.5324 - acc: 0.7460 - val_loss: 0.3997 - val_acc: 0.7581

Epoch 00001: val_acc improved from -inf to 0.75813, saving model to model_best_checkpoint.h5
Epoch 2/400
150/150 [==============================] - 4s 24ms/step - loss: 0.3427 - acc: 0.8556 - val_loss: 0.3493 - val_acc: 0.8644

Epoch 00002: val_acc improved from 0.75813 to 0.86438, saving model to model_best_checkpoint.h5
Epoch 3/400
150/150 [==============================] - 4s 25ms/step - loss: 0.2432 - acc: 0.9090 - val_loss: 0.3117 - val_acc: 0.8862

Epoch 00003: val_acc improved from 0.86438 to 0.88625, saving model to model_best_checkpoint.h5
Epoch 4/400
150/150 [==============================] - 4s 25ms/step - loss: 0.2382 - acc: 0.9113 - val_loss: 0.2826 - val_acc: 0.9115

Epoch 00004: val_acc improved from 0.88625 to 0.91146, saving model to model_best_checkpoint.h5
Epoch 5/400
150/150 [==============================] - 4s 26ms/step - loss: 0.2


Epoch 00042: val_acc did not improve from 0.96051
Epoch 43/400
150/150 [==============================] - 4s 25ms/step - loss: 0.1342 - acc: 0.9437 - val_loss: 0.1942 - val_acc: 0.9185

Epoch 00043: val_acc did not improve from 0.96051
Epoch 44/400
150/150 [==============================] - 4s 25ms/step - loss: 0.1139 - acc: 0.9586 - val_loss: 0.1285 - val_acc: 0.9519

Epoch 00044: val_acc did not improve from 0.96051
Epoch 45/400
150/150 [==============================] - 4s 25ms/step - loss: 0.1261 - acc: 0.9504 - val_loss: 0.1747 - val_acc: 0.9287

Epoch 00045: val_acc did not improve from 0.96051
Epoch 46/400
150/150 [==============================] - 4s 24ms/step - loss: 0.1332 - acc: 0.9519 - val_loss: 0.1118 - val_acc: 0.9550

Epoch 00046: val_acc did not improve from 0.96051
Epoch 47/400
150/150 [==============================] - 4s 25ms/step - loss: 0.1168 - acc: 0.9531 - val_loss: 0.1067 - val_acc: 0.9561

Epoch 00047: val_acc did not improve from 0.96051
Epoch 48/400
150/15

150/150 [==============================] - 4s 25ms/step - loss: 0.1010 - acc: 0.9596 - val_loss: 0.1091 - val_acc: 0.9580

Epoch 00086: val_acc did not improve from 0.98438
Epoch 87/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0868 - acc: 0.9681 - val_loss: 0.1059 - val_acc: 0.9581

Epoch 00087: val_acc did not improve from 0.98438
Epoch 88/400
150/150 [==============================] - 4s 24ms/step - loss: 0.0958 - acc: 0.9625 - val_loss: 0.1029 - val_acc: 0.9575

Epoch 00088: val_acc did not improve from 0.98438
Epoch 89/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0892 - acc: 0.9671 - val_loss: 0.0741 - val_acc: 0.9725

Epoch 00089: val_acc did not improve from 0.98438
Epoch 90/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0976 - acc: 0.9632 - val_loss: 0.0883 - val_acc: 0.9650

Epoch 00090: val_acc did not improve from 0.98438
Epoch 91/400
150/150 [==============================] - 4s 26ms/step - loss: 0.0865


Epoch 00129: val_acc did not improve from 0.98500
Epoch 130/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0769 - acc: 0.9704 - val_loss: 0.0491 - val_acc: 0.9781

Epoch 00130: val_acc did not improve from 0.98500
Epoch 131/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0807 - acc: 0.9700 - val_loss: 0.0580 - val_acc: 0.9775

Epoch 00131: val_acc did not improve from 0.98500
Epoch 132/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0671 - acc: 0.9767 - val_loss: 0.0487 - val_acc: 0.9841

Epoch 00132: val_acc did not improve from 0.98500
Epoch 133/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0714 - acc: 0.9727 - val_loss: 0.0837 - val_acc: 0.9637

Epoch 00133: val_acc did not improve from 0.98500
Epoch 134/400
150/150 [==============================] - 4s 24ms/step - loss: 0.0730 - acc: 0.9704 - val_loss: 0.0656 - val_acc: 0.9719

Epoch 00134: val_acc did not improve from 0.98500
Epoch 135/400


150/150 [==============================] - 4s 26ms/step - loss: 0.0657 - acc: 0.9760 - val_loss: 0.0294 - val_acc: 0.9906

Epoch 00173: val_acc improved from 0.99000 to 0.99062, saving model to model_best_checkpoint.h5
Epoch 174/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0669 - acc: 0.9742 - val_loss: 0.0324 - val_acc: 0.9881

Epoch 00174: val_acc did not improve from 0.99062
Epoch 175/400
150/150 [==============================] - 4s 26ms/step - loss: 0.0652 - acc: 0.9754 - val_loss: 0.0852 - val_acc: 0.9662

Epoch 00175: val_acc did not improve from 0.99062
Epoch 176/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0704 - acc: 0.9746 - val_loss: 0.0460 - val_acc: 0.9800

Epoch 00176: val_acc did not improve from 0.99062
Epoch 177/400
150/150 [==============================] - 4s 24ms/step - loss: 0.0654 - acc: 0.9735 - val_loss: 0.0800 - val_acc: 0.9725

Epoch 00177: val_acc did not improve from 0.99062
Epoch 178/400
150/150 [==========

150/150 [==============================] - 4s 25ms/step - loss: 0.0535 - acc: 0.9821 - val_loss: 0.0478 - val_acc: 0.9788

Epoch 00216: val_acc did not improve from 0.99187
Epoch 217/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0659 - acc: 0.9748 - val_loss: 0.0348 - val_acc: 0.9869

Epoch 00217: val_acc did not improve from 0.99187
Epoch 218/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0543 - acc: 0.9808 - val_loss: 0.0811 - val_acc: 0.9650

Epoch 00218: val_acc did not improve from 0.99187
Epoch 219/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0586 - acc: 0.9773 - val_loss: 0.0583 - val_acc: 0.9788

Epoch 00219: val_acc did not improve from 0.99187
Epoch 220/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0564 - acc: 0.9792 - val_loss: 0.0278 - val_acc: 0.9919

Epoch 00220: val_acc did not improve from 0.99187
Epoch 221/400
150/150 [==============================] - 4s 25ms/step - loss: 0

150/150 [==============================] - 4s 25ms/step - loss: 0.0555 - acc: 0.9796 - val_loss: 0.0299 - val_acc: 0.9919

Epoch 00259: val_acc did not improve from 0.99375
Epoch 260/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0514 - acc: 0.9815 - val_loss: 0.0470 - val_acc: 0.9838

Epoch 00260: val_acc did not improve from 0.99375
Epoch 261/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0514 - acc: 0.9815 - val_loss: 0.0462 - val_acc: 0.9822

Epoch 00261: val_acc did not improve from 0.99375
Epoch 262/400
150/150 [==============================] - 4s 26ms/step - loss: 0.0540 - acc: 0.9792 - val_loss: 0.0547 - val_acc: 0.9794

Epoch 00262: val_acc did not improve from 0.99375
Epoch 263/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0472 - acc: 0.9822 - val_loss: 0.0296 - val_acc: 0.9906

Epoch 00263: val_acc did not improve from 0.99375
Epoch 264/400
150/150 [==============================] - 4s 25ms/step - loss: 0

150/150 [==============================] - 4s 25ms/step - loss: 0.0431 - acc: 0.9846 - val_loss: 0.0274 - val_acc: 0.9912

Epoch 00302: val_acc did not improve from 0.99500
Epoch 303/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0512 - acc: 0.9827 - val_loss: 0.0303 - val_acc: 0.9919

Epoch 00303: val_acc did not improve from 0.99500
Epoch 304/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0510 - acc: 0.9815 - val_loss: 0.0250 - val_acc: 0.9904

Epoch 00304: val_acc did not improve from 0.99500
Epoch 305/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0570 - acc: 0.9792 - val_loss: 0.0632 - val_acc: 0.9769

Epoch 00305: val_acc did not improve from 0.99500
Epoch 306/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0507 - acc: 0.9821 - val_loss: 0.0174 - val_acc: 0.9962

Epoch 00306: val_acc improved from 0.99500 to 0.99625, saving model to model_best_checkpoint.h5
Epoch 307/400
150/150 [==========


Epoch 00345: val_acc did not improve from 0.99625
Epoch 346/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0460 - acc: 0.9837 - val_loss: 0.0429 - val_acc: 0.9850

Epoch 00346: val_acc did not improve from 0.99625
Epoch 347/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0448 - acc: 0.9858 - val_loss: 0.0360 - val_acc: 0.9892

Epoch 00347: val_acc did not improve from 0.99625
Epoch 348/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0554 - acc: 0.9802 - val_loss: 0.0419 - val_acc: 0.9856

Epoch 00348: val_acc did not improve from 0.99625
Epoch 349/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0427 - acc: 0.9837 - val_loss: 0.0344 - val_acc: 0.9881

Epoch 00349: val_acc did not improve from 0.99625
Epoch 350/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0425 - acc: 0.9841 - val_loss: 0.0539 - val_acc: 0.9847

Epoch 00350: val_acc did not improve from 0.99625
Epoch 351/400



Epoch 00389: val_acc did not improve from 0.99625
Epoch 390/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0397 - acc: 0.9869 - val_loss: 0.0381 - val_acc: 0.9866

Epoch 00390: val_acc did not improve from 0.99625
Epoch 391/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0374 - acc: 0.9864 - val_loss: 0.0297 - val_acc: 0.9894

Epoch 00391: val_acc did not improve from 0.99625
Epoch 392/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0398 - acc: 0.9850 - val_loss: 0.0265 - val_acc: 0.9906

Epoch 00392: val_acc did not improve from 0.99625
Epoch 393/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0465 - acc: 0.9823 - val_loss: 0.0313 - val_acc: 0.9904

Epoch 00393: val_acc did not improve from 0.99625
Epoch 394/400
150/150 [==============================] - 4s 25ms/step - loss: 0.0410 - acc: 0.9852 - val_loss: 0.0195 - val_acc: 0.9938

Epoch 00394: val_acc did not improve from 0.99625
Epoch 395/400


In [34]:
tests =  glob.glob('../input/test/test/*')
imgs = np.array([ img_to_array( load_img(file, target_size = IMG_DIM) ) for file in tests ])/255
submission = pd.DataFrame({'id':tests})
submission.id = submission.id.apply(lambda x: x.split('/')[-1])
submission.head()

,id
0,c662bde123f0f83b3caae0ffda237a93.jpg
1,9553eed7793d4cf88b5226d446d93dae.jpg
2,19f059a7ce41b25be1548bc4049b45ec.jpg
3,fb4f464486f4894330273346ce939252.jpg
4,b52558a522db6ec2501ae188b6d6e526.jpg


In [37]:
submission['has_cactus'] = np.squeeze(model.predict(imgs))
submission.to_csv('samplesubmission.csv',index=False)
submission.head()

,id,has_cactus
0,c662bde123f0f83b3caae0ffda237a93.jpg,0.999899
1,9553eed7793d4cf88b5226d446d93dae.jpg,0.000865
2,19f059a7ce41b25be1548bc4049b45ec.jpg,1.000000
3,fb4f464486f4894330273346ce939252.jpg,1.000000
4,b52558a522db6ec2501ae188b6d6e526.jpg,0.999114
